# Chocolate Sales Data
In this notebook I will do an exploratory data analysis (EDA) for chocolate sales data and later discover how I will leverage the data to make business insights, predictions or any notable findings. Quoting from the author of the data, here are some notes regarding the data collection methodology:
- Data was aggregated from chocolate retailers and online marketplaces.
- Only confirmed transactions were included to ensure accuracy.
- Revenue values reflect final prices after applying discounts, if any.

*Source*: [Chocolate Sales Data 📊🍫](https://www.kaggle.com/datasets/atharvasoundankar/chocolate-sales/data), authored by: [Atharva Soundankar](https://www.kaggle.com/atharvasoundankar)

### Step 1: Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re

# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression
# from sklearn.pipeline import make_pipeline
# import warnings

### Step 2: Read the CSV File and Import the Data as a Pandas Dataframe

In [2]:
df = pd.read_csv('/home/jasonzelin/data-analytics-portfolio/chocolate_sales_data/data/chocolate_sales.csv')

### Step 3: Start the Exploratory Data Analysis (EDA)

Identify the shape and data types of each column existing in the dataframe

In [3]:
print("The shape of the data is ", df.shape)
print("The data types of each column are as below:", df.dtypes)

The shape of the data is  (1094, 6)
The data types of each column are as below: Sales Person     object
Country          object
Product          object
Date             object
Amount           object
Boxes Shipped     int64
dtype: object
